In [1]:
import os
import re
import collections
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import pymorphy2
import json
morph = pymorphy2.MorphAnalyzer()
curr_dir = os.getcwd()
curr_dir = curr_dir + '/friends-data'
word_pattern = re.compile('\w+')


In [3]:
def count_every_word(directory_name):
    c = collections.Counter()
    # смотрю все документы
    # можно было бы вынести это в отдельную функцию просмотр
    # документов, но я этого не сделал
    for root, dirs, files in os.walk(directory_name):
        for name in files:
            fpath = os.path.join(root, name)
            print(fpath)
            # не смотрю на системные папки
            if fpath[-9:] == '.DS_Store':
                continue
            # читаю по очереди все тексты субтитров
            with open(fpath, 'r') as f:
                text = f.read()
                # в тексте субтитров выделяю слова регулярными выражениями
                text_words = re.findall(word_pattern, text.lower())
                # все слова собираю в единый список
                for word in text_words:
                    c[word] += 1
    return dict(c)

In [4]:
def index_for_words(directory_name, dict_of_words):
    episode_name_pattern = re.compile('(Friends - \d.*)')
    # то же самое, что и в прошлой функции
    # до следующего комментария: иду по текстам
    for key in dict_of_words:
        dict_of_words[key] = []
    for root, dirs, files in os.walk(directory_name):
        for name in files:
            fpath = os.path.join(root, name)
            print(fpath)
            if fpath[-9:] == '.DS_Store':
                continue
            with open(fpath, 'r') as f:
                text = f.read()
                # вот до сюда то же самое,
                # здесь я зайдя в каждый текст проверяю есть ли в нем это слово
                for key in dict_of_words:
                    if key in text.lower():
                        dict_of_words[key].append(re.search(
                            episode_name_pattern, fpath).group(0))
    print(dict_of_words)
    with open('yatsishin_hw1.json', 'w') as f:
        json.dump(dict_of_words, f)


In [5]:
index_for_words(curr_dir, count_every_word(curr_dir))

/Users/egor/Documents/GitHub/infopoisk/HW1_infopoisk/friends-data/.DS_Store
/Users/egor/Documents/GitHub/infopoisk/HW1_infopoisk/friends-data/Friends - season 2/Friends - 2x08 - The One With The List.ru.txt
/Users/egor/Documents/GitHub/infopoisk/HW1_infopoisk/friends-data/Friends - season 2/Friends - 2x20 - The One Where Old Yeller Dies.NurlanB.ru.txt
/Users/egor/Documents/GitHub/infopoisk/HW1_infopoisk/friends-data/Friends - season 2/Friends - 2x24 - The One With Barry And Mindy's Wedding.ru.txt
/Users/egor/Documents/GitHub/infopoisk/HW1_infopoisk/friends-data/Friends - season 2/Friends - 2x16 - The One Where Joey Moves Out.ru.txt
/Users/egor/Documents/GitHub/infopoisk/HW1_infopoisk/friends-data/Friends - season 2/Friends - 2x07 - The One Where Ross Finds Out.ru.txt
/Users/egor/Documents/GitHub/infopoisk/HW1_infopoisk/friends-data/Friends - season 2/Friends - 2x17 - The One Where Eddie Moves In.ru.txt
/Users/egor/Documents/GitHub/infopoisk/HW1_infopoisk/friends-data/Friends - season 2

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [2]:
def index_td(directory_name):
    # инициализация
    corpus = []
    vectorizer = CountVectorizer()
    for root, dirs, files in os.walk(directory_name):
        for name in files:
            fpath = os.path.join(root, name)
            print(fpath)
            # не смотрю на системные папки
            if fpath[-9:] == '.DS_Store':
                continue
            # читаю по очереди все тексты субтитров
            with open(fpath, 'r') as f:
                text = f.read()
                corpus.append(text.lower())
        # для получения самого популярного персонажа
        # понадобится лемматизация, а для pymorphy2
        # нужно поделить текст на слова
        for entry in corpus:
            list_from_text = re.findall(word_pattern, entry)
            print(list_from_text)
            # выше поделил текст, теперь по каждому слову нужно
            # пройти pymorphy2
            for word in list_from_text:
                word = morph.parse(word)[0].normal_form
            # соединяю лемматизированные слова
            # обратно в текст через пробел
            entry = ' '.join(list_from_text)
    # считаем матрицу
    x = vectorizer.fit_transform(corpus)
    df = pd.DataFrame(x.todense(), columns=vectorizer.get_feature_names())
    df.to_pickle("./yatsishin_hw1.pkl")

In [2]:
index_td(curr_dir)

NameError: name 'curr_dir' is not defined

In [5]:
df = pd.read_pickle("yatsishin_hw1.pkl")

In [7]:
# ответы на вопросы задания
print(f'Самое частотное слово - {df.sum(axis=0).idxmax()}')
print(f'Самое редкое слово - {df.sum(axis=0).idxmin()}')

mask = (df != 0).all(axis=0)
print(f'Слова, которые есть во всех документах: {", ".join(df.columns[mask])}')

Самое частое слово - не
Самое редкое слово - 03815
Самое редкое слово, если удалить числа и таймкоды - 1986ом
Слова, которые есть во всех документах: да, если, как, меня, мне, мы, на, не, нет, но, ну, он, она, просто, так, тебе, тебя, то, ты, что, это


In [14]:
spellings = {'Чендлер':['чендлер','чендлера','чэндлер'],'Моника':['моника'], 'Фиби':['фиби','pheebs','фибc','фибс'], 'Рейчел':['rachel','рейч'], 'Росс': ['росс','ross'],'Джоуи':['джоуи','джоу']}
for caracter, names in spellings.items():
    df[caracter] = df[names].sum(axis=1)

print(f"Самый популярный персонаж без разных написаний - {df[['рейч','моника','росс','джоуи','фиби','чендлер']].sum(axis=0).idxmax()}")
print(f"Самый популярный персонаж с разными написаниями - {df[['Рейчел','Моника','Росс','Джоуи','Фиби','Чендлер']].sum(axis=0).idxmax()}")

Самый популярный персонаж без разных написаний - росс
Самый популярный персонаж с разными написаниями - Фиби
